In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
import math


df = pd.read_csv("/content/xy_data (1).csv", header=None, names=["x", "y"], dtype=str)
print("RAW head:\n", df.head())
print("RAW dtypes:\n", df.dtypes)


df['x'] = pd.to_numeric(df['x'].str.strip().str.replace(',', ''), errors='coerce')
df['y'] = pd.to_numeric(df['y'].str.strip().str.replace(',', ''), errors='coerce')

print("\nAfter coercion head:\n", df.head())
print("After coercion dtypes:\n", df.dtypes)
print("Number of NaNs in x,y:", df['x'].isna().sum(), df['y'].isna().sum())


df = df.dropna(subset=['x', 'y']).reset_index(drop=True)
xs = df['x'].values.astype(float)
ys = df['y'].values.astype(float)
N = len(xs)
print("Using N =", N, "rows after cleaning.")

if N < 2:
    raise RuntimeError("Not enough numeric rows after cleaning the CSV.")

RAW head:
            x          y
0          x          y
1  88.364456  57.784378
2  74.283936   54.40678
3  60.256474  46.311462
4   82.13437  57.717567
RAW dtypes:
 x    object
y    object
dtype: object

After coercion head:
            x          y
0        NaN        NaN
1  88.364456  57.784378
2  74.283936  54.406780
3  60.256474  46.311462
4  82.134370  57.717567
After coercion dtypes:
 x    float64
y    float64
dtype: object
Number of NaNs in x,y: 1 1
Using N = 1500 rows after cleaning.


In [3]:
theta_bounds_deg = (1e-6, 50.0 - 1e-6)
M_bounds = (-0.05 + 1e-12, 0.05 - 1e-12)
X_bounds = (0.0, 100.0)

t_min, t_max = 6.0, 60.0
t = np.linspace(t_min, t_max, N)



In [4]:

def model_coords(params, tvals):
    theta_deg, M, X = params
    theta = np.deg2rad(float(theta_deg))
    exp_term = np.exp(float(M) * np.abs(tvals))
    x_model = tvals * np.cos(theta) - exp_term * np.sin(0.3 * tvals) * np.sin(theta) + float(X)
    y_model = 42.0 + tvals * np.sin(theta) + exp_term * np.sin(0.3 * tvals) * np.cos(theta)
    return x_model, y_model

def residuals(params, tvals, x_obs, y_obs):
    x_model, y_model = model_coords(params, tvals)
    return np.concatenate([x_model - x_obs, y_model - y_obs])

lower = [theta_bounds_deg[0], M_bounds[0], X_bounds[0]]
upper = [theta_bounds_deg[1], M_bounds[1], X_bounds[1]]

initial_guesses = [
    [20.0, 0.0, 10.0],
    [10.0, 0.01, 20.0],
    [30.0, -0.01, 5.0],
    [5.0, 0.02, 50.0],
    [40.0, -0.02, 80.0]
]

In [5]:

best = None
best_cost = 1e99
for guess in initial_guesses:
    try:
        res = least_squares(residuals, x0=guess, args=(t, xs, ys),
                            bounds=(lower, upper), verbose=0,
                            ftol=1e-9, xtol=1e-9, gtol=1e-9,
                            loss='huber')
    except Exception as e:
        print("optimizer failed for guess", guess, ":", e)
        continue
    print("guess", guess, "-> cost", res.cost)
    if res.cost < best_cost:
        best_cost = res.cost
        best = res

if best is None:
    raise RuntimeError("All optimization attempts failed (after cleaning)")

theta_deg_fit, M_fit, X_fit = best.x
print("Fitted θ (deg) =", theta_deg_fit, " M =", M_fit, " X =", X_fit)


guess [20.0, 0.0, 10.0] -> cost 36398.984189287294
guess [10.0, 0.01, 20.0] -> cost 36398.98418926004
guess [30.0, -0.01, 5.0] -> cost 36398.98418937065
guess [5.0, 0.02, 50.0] -> cost 36398.984189263734
guess [40.0, -0.02, 80.0] -> cost 36398.98418930346
Fitted θ (deg) = 27.987638910395663  M = 0.02103826976311748  X = 54.865039463878595


In [6]:

x_fit, y_fit = model_coords(best.x, t)

print("Fitted parameters:")
print(f"  θ (deg) = {theta_deg_fit:.8f}")
print(f"  M       = {M_fit:.12f}")
print(f"  X       = {X_fit:.8f}")
print("Final cost (sum of squares / 2) =", best.cost)
# compute L1 sum between predicted and observed (the assessment metric is L1)
L1 = np.sum(np.abs(x_fit - xs) + np.abs(y_fit - ys))
print("Total L1 (sum |dx|+|dy|) =", L1)




Fitted parameters:
  θ (deg) = 27.98763891
  M       = 0.021038269763
  X       = 54.86503946
Final cost (sum of squares / 2) = 36398.98418926004
Total L1 (sum |dx|+|dy|) = 37865.67977484637


In [7]:
theta_rad = math.radians(theta_deg_fit)
latex_expr = (r"\left("
              f"t*\\cos({theta_deg_fit:.6f}^\\circ) - e^{{{M_fit:.8f}|t|}} \\cdot \\sin(0.3t) \\sin({theta_deg_fit:.6f}^\\circ) + {X_fit:.6f}, "
              f"\\; 42 + t*\\sin({theta_deg_fit:.6f}^\\circ) + e^{{{M_fit:.8f}|t|}} \\cdot \\sin(0.3t) \\cos({theta_deg_fit:.6f}^\\circ)"
              r"\right)")
print("\nLaTeX parametric expression (ready for readme/desmos):")
print(latex_expr)





LaTeX parametric expression (ready for readme/desmos):
\left(t*\cos(27.987639^\circ) - e^{0.02103827|t|} \cdot \sin(0.3t) \sin(27.987639^\circ) + 54.865039, \; 42 + t*\sin(27.987639^\circ) + e^{0.02103827|t|} \cdot \sin(0.3t) \cos(27.987639^\circ)\right)


In [8]:
out = pd.DataFrame({"t":t, "x_model":x_fit, "y_model":y_fit})
out.to_csv("predicted_curve.csv", index=False)
print("Predicted curve saved to predicted_curve.csv")

Predicted curve saved to predicted_curve.csv
